In [9]:
INPUT_FOLDER = '/Users/bober/Desktop/study/naukma/prog/data'

INPUT_FILE_MERGED = 'merged_dataset_v1.csv'
INPUT_FILE_ISW = 'isw_all_days_parsed_v1.csv'
INPUT_FOLDER = '/Users/bober/Desktop/study/naukma/prog/data'

MODEL_FOLDER = "models"


tfidf_transformer_model = "tfidf_transformer"
count_vectorizer_model = "count_vectorizer"

tfidf_transformer_version = "v1"
count_vectorizer_version = "v1"

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [11]:
import pandas as pd
import datetime
import pickle
import scipy
import numpy as np

In [12]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words
from nltk.corpus import stopwords
import nltk

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [14]:
df_raw_data = pd.read_csv(f'{INPUT_FOLDER}/{INPUT_FILE_MERGED}',sep = ';')

In [15]:
df_raw_data

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,...,event_end_date,event_intersection_alarm_region_counts,event_amount_of_alarms_for_one_day,event_start_hour,event_end_hour,event_last_24_hour_in_region,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch
0,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194527,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194528,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194529,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194530,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#df_raw_data[df_raw_data['day_datetime']=='2022-04-28']

In [17]:
is_alarm = ~((df_raw_data['event_end_hour'].isna()) & (df_raw_data['event_start_hour'].isna()))

In [18]:
df_raw_data['is_alarm']= is_alarm
df_raw_data

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,...,event_intersection_alarm_region_counts,event_amount_of_alarms_for_one_day,event_start_hour,event_end_hour,event_last_24_hour_in_region,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch,is_alarm
0,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194527,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194528,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194529,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194530,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [19]:
df_raw_data.describe()


,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,...,event_region_id,event_all_region,event_intersection_alarm_id,event_duration_in_mins_int,event_intersection_alarm_region_counts,event_amount_of_alarms_for_one_day,event_last_24_hour_in_region,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_datetimeEpoch
count,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,194532.000000,...,49249.000000,49249.000000,4924.000000,49249.000000,49249.000000,49249.000000,49249.000000,4.924900e+04,4.924900e+04,4.924900e+04
mean,15.298069,6.714788,10.961227,5.256789,71.500915,2.501287,7.093413,154.529203,13.310194,5.317917,...,12.518508,0.755569,14621.973193,78.500416,12.075859,85.680562,6.331682,1.657251e+09,1.657259e+09,1.657255e+09
std,9.911493,7.898592,8.762649,7.516936,15.056165,15.227688,12.344893,100.067642,8.663774,2.781952,...,6.290175,0.429754,13597.299433,84.982098,7.650067,46.838996,6.037332,8.264011e+06,8.263888e+06,8.263950e+06
min,-11.700000,-50.300000,-17.200000,-20.600000,23.300000,0.000000,0.000000,1.100000,0.000000,0.000000,...,1.000000,0.000000,45.000000,0.000000,1.000000,1.000000,1.000000,1.645826e+09,1.645834e+09,1.645826e+09
25%,7.200000,0.300000,3.800000,-0.200000,59.900000,0.000000,0.000000,60.400000,5.100000,3.000000,...,8.000000,1.000000,3382.000000,29.000000,5.000000,50.000000,3.000000,1.649290e+09,1.649293e+09,1.649293e+09
50%,15.800000,6.700000,11.000000,5.800000,72.200000,0.074000,4.170000,150.000000,12.900000,6.000000,...,13.000000,1.000000,8376.000000,50.000000,10.000000,72.000000,5.000000,1.656072e+09,1.656079e+09,1.656072e+09
75%,23.200000,13.000000,18.200000,11.100000,84.400000,1.100000,8.330000,235.400000,20.300000,8.000000,...,18.000000,1.000000,23266.000000,96.000000,21.000000,113.000000,7.000000,1.664507e+09,1.664510e+09,1.664510e+09
max,37.000000,24.200000,30.100000,21.100000,100.000000,306.000000,83.330000,359.500000,30.900000,10.000000,...,24.000000,1.000000,47488.000000,1123.000000,23.000000,198.000000,44.000000,1.674230e+09,1.674234e+09,1.674234e+09


In [20]:
df_isw_raw = pd.read_csv(f'{INPUT_FOLDER}/{INPUT_FILE_ISW}',sep = ';')
df_isw_raw

,date,short_url,title,text_title,full_url,main_html,main_html_v2,main_html_v7,report_text_lemm,report_text_stemm,keywords
0,2022-02-24,russia_ukraine_warning_update_initial_russian_...,Russia-Ukraine Warning Update: Initial Russian...,Russia-Ukraine Warning Update: Initial Russian...,/backgrounder/russia-ukraine-warning-update-in...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nRussian President Vladimir Putin began a lar...,russian presid vladimir putin began larg scal...,russian presid vladimir putin began larg scal...,"{'pm': 0.388, 'airport': 0.262, 'kyiv': 0.244,..."
1,2022-02-25,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",\nRussian forces entered the outskirts of Kyiv...,russian forc enter outskirt kyiv west bank dn...,russian forc enter outskirt kyiv west bank dn...,"{'pm': 0.36, 'zero': 0.351, 'februari': 0.336,..."
2,2022-02-27,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna St...",\nThe Russian military has likely recognized t...,russian militari like recogn initi expect lim...,russian militari like recogn initi expect lim...,"{'februari': 0.553, 'kyiv': 0.262, 'seven': 0...."
3,2022-02-28,russian_offensive_campaign_assessment_february...,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nThe Russian military is reorganizing its mil...,russian militari reorgan militari effort atte...,russian militari reorgan militari effort atte...,"{'februari': 0.593, 'eight': 0.267, 'kyiv': 0...."
4,2022-03-01,russian_offensive_campaign_assessment_march_1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nFrederick W. Kagan, George Barros, and Kat...",\nRussian forces are completing the reinforcem...,russian forc complet reinforc resuppli troop ...,russian forc complet reinforc resuppli troop ...,"{'kyiv': 0.357, 'envelop': 0.287, 'chernihiv':..."
...,...,...,...,...,...,...,...,...,...,...,...
329,2023-01-21,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nKarolina Hird, Grace Mappes, Angela Howard...",\nThe Ukrainian defense of Bakhmut is likely a...,ukrainian defens bakhmut like strateg sound e...,ukrainian defen bakhmut like strateg sound ef...,"{'januari': 0.491, 'teplinski': 0.219, 'milblo..."
330,2023-01-22,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, Januar...","Russian Offensive Campaign Assessment, Januar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nRussian Offensive Campaign Assessment, Janu...",\nISW is publishing an abbreviated campaign up...,isw publish abbrevi campaign updat today janu...,isw publish abbrevi campaign updat today janu...,"{'prigozhin': 0.6, 'putin': 0.328, 'wagner': 0..."
331,2023-01-23,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-cam

### dropping the row and selecting the necessary columns/


In [21]:
#pd.to_datetime(df_isw_raw['date'])+datetime.timedelta(days=1)
df_isw_raw['report_date']=df_isw_raw['date']
df_isw_raw['date_tmrw']= (pd.to_datetime(df_isw_raw['date'])+datetime.timedelta(days=1)).dt.strftime('%Y-%m-%d')


In [22]:
df_isw_upd = df_isw_raw.drop( labels = 0, axis=0,inplace=False)[["report_date", "date_tmrw", "keywords", "main_html_v7", "report_text_lemm"]].copy().add_prefix('isw_')


In [23]:
df_isw_upd

,isw_report_date,isw_date_tmrw,isw_keywords,isw_main_html_v7,isw_report_text_lemm
1,2022-02-25,2022-02-26,"{'pm': 0.36, 'zero': 0.351, 'februari': 0.336,...",\nRussian forces entered the outskirts of Kyiv...,russian forc enter outskirt kyiv west bank dn...
2,2022-02-27,2022-02-28,"{'februari': 0.553, 'kyiv': 0.262, 'seven': 0....",\nThe Russian military has likely recognized t...,russian militari like recogn initi expect lim...
3,2022-02-28,2022-03-01,"{'februari': 0.593, 'eight': 0.267, 'kyiv': 0....",\nThe Russian military is reorganizing its mil...,russian militari reorgan militari effort atte...
4,2022-03-01,2022-03-02,"{'kyiv': 0.357, 'envelop': 0.287, 'chernihiv':...",\nRussian forces are completing the reinforcem...,russian forc complet reinforc resuppli troop ...
5,2022-03-02,2022-03-03,"{'march': 0.546, 'kyiv': 0.382, 'envelop': 0.2...",\nRussian forces resumed offensive operations ...,russian forc resum offens oper support envelo...
...,...,...,...,...,...
329,2023-01-21,2023-01-22,"{'januari': 0.491, 'teplinski': 0.219, 'milblo...",\nThe Ukrainian defense of Bakhmut is likely a...,ukrainian defens bakhmut like strateg sound e...
330,2023-01-22,2023-01-23,"{'prigozhin': 0.6, 'putin': 0.328, 'wagner': 0...",\nISW is publishing an abbreviated campaign up...,isw publish abbrevi campaign updat today janu...
331,2023-01-23,2023-01-24,"{'januari': 0.679, 'three': 0.171, 'two': 0.13...",\nUkrainian intelligence assessed that Russian...,ukrainian intellig ass russian forc prepar of...
332,2023-01-24,2023-01-25,"{'januari': 0.516, 'claim': 0.184, 'bakhmut': ...",\nA coalition of NATO member states reportedly...,coalit nato member state reportedli send ukra...


In [24]:
df_all = df_raw_data.merge(df_isw_upd, how = 'left', left_on = 'day_datetime', right_on = 'isw_date_tmrw')

In [25]:
df_all.sample(5)

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,...,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch,is_alarm,isw_report_date,isw_date_tmrw,isw_keywords,isw_main_html_v7,isw_report_text_lemm
57147,"Київ, Україна",2022-08-29,31.2,16.4,23.7,13.8,59.3,4.0,4.17,238.2,...,NaN,NaN,NaN,NaN,False,2022-08-28,2022-08-29,"{'august': 0.328, 'beglov': 0.278, 'citi': 0.1...",\n Russian President Vladimir Putin signed two...,russian presid vladimir putin sign two decre ...
93764,"Харків, Україна",2022-12-20,0.0,-9.0,-4.2,-8.6,73.6,0.0,0.00,40.1,...,NaN,NaN,NaN,NaN,False,2022-12-19,2022-12-20,"{'decemb': 0.526, 'lukashenko': 0.205, 'milblo...",\n Belarusian President Alexander Lukashenko l...,belarusian presid alexand lukashenko like def...
74820,"Чернігів, Україна",2022-09-25,13.9,6.2,10.0,5.7,76.1,0.0,0.00,148.0,...,NaN,NaN,NaN,NaN,False,2022-09-24,2022-09-25,"{'mobil': 0.301, 'commissar': 0.255, 'septemb'...",\nRussian Defense Minister Sergei Shoigu’s dec...,russian defens minist sergei shoigu declar ca...
49902,"Дніпро, Україна",2023-01-09,-3.2,-12.8,-8.5,-14.7,62.1,0.0,0.00,64.7,...,1.673258e+09,1.673266e+09,2023-01-09 10:00:00,1.673258e+09,True,2023-01-08,2023-01-09,"{'januari': 0.504, 'mod': 0.26, 'prigozhin': 0...",\n ISW is publishing an abbreviated campaign u...,isw publish abbrevi campaign updat today janu...
191996,"Луцьк, Луцький район, Україна",2022-10-07,17.6,5.6,12.0,8.1,78.9,0.0,0.00,146.8,...,NaN,NaN,NaN,NaN,False,2022-10-06,2022-10-07,"{'octob': 0.474, 'mobil': 0.202, 'kupyansk': 0...",\nRussia’s use of Iranian-made drones is not g...,russia use iranian made drone not gener asymm...


In [26]:
df_all

,city_resolvedAddress,day_datetime,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,...,event_start_hour_datetimeEpoch,event_end_hour_datetimeEpoch,event_hour_level_event_time,event_hour_level_event_datetimeEpoch,is_alarm,isw_report_date,isw_date_tmrw,isw_keywords,isw_main_html_v7,isw_report_text_lemm
0,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
1,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
2,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
3,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
4,"Херсон, Україна",2022-02-25,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194527,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,False,2023-01-19,2023-01-20,"{'januari': 0.427, 'prigozhin': 0.213, 'belaru...",\nSenior Kremlin officials continue holding hi...,senior kremlin offici continu hold high level...
194528,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,False,2023-01-19,2023-01-20,"{'januari': 0.427, 'prigozhin': 0.213, 'belaru...",\nSenior Kremlin officials continue holding hi...,senior kremlin offici continu hold high level...
194529,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,False,2023-01-19,2023-01-20,"{'januari': 0.427, 'prigozhin': 0.213, 'belaru...",\nSenior Kremlin officials continue holding hi...,senior kremlin offici continu hold high level...
194530,"Луцьк, Луцький район, Україна",2023-01-20,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,...,NaN,NaN,NaN,NaN,False,2023-01-19,2023-01-20,"{'januari': 0.427, 'prigozhin': 0.213, 'belaru...",\nSenior Kremlin officials continue holding hi...,senior kremlin offici continu hold high level...


In [27]:
to_delete = ['city_resolvedAddress',
'day_datetime',
"hour_datetime",
"hour_datetimeEpoch",
"city",
"region",
"center_city_ua",
"center_city_en",
"event_region_title",
"event_region_city",
"event_all_region",
"event_clean_end",
"event_intersection_alarm_id",
"event_start_hour",
"event_end_hour",
"event_start_hour_datetimeEpoch",
"event_end_hour_datetimeEpoch",
"event_hour_level_event_time",
"event_hour_level_event_datetimeEpoch",
"isw_report_date",
'isw_date_tmrw',
'isw_main_html_v7',
'isw_keywords',
'isw_report_text_lemm'
]

In [28]:
df_weather_matrix = df_all.drop(to_delete, axis = 1)
df_weather_matrix

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,...,event_start,event_end,event_duration,event_duration_in_mins_int,event_start_date,event_end_date,event_intersection_alarm_region_counts,event_amount_of_alarms_for_one_day,event_last_24_hour_in_region,is_alarm
0,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,8.7,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,8.7,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,8.7,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,8.7,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,6.4,0.7,3.7,0.2,78.0,0.000,0.00,99.5,8.7,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194527,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,1.6,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194528,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,1.6,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194529,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,1.6,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
194530,3.7,0.3,1.5,0.4,92.5,0.704,4.17,19.6,1.6,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [29]:
additional_to_delete = [
"day_sunset",
"hour_preciptype",
"hour_conditions",
"region_alt",
"event_start",
"event_end",
"hour_solarenergy",
'event_region_id',
'event_duration',
'event_duration_in_mins_int',
'event_amount_of_alarms_for_one_day',
'event_intersection_alarm_region_counts',
'event_last_24_hour_in_region',
'event_start_date',
'event_end_date',
'day_sunrise'
]

In [30]:
df_weather_matrix_v1 = df_weather_matrix.drop(additional_to_delete, axis = 1)


In [31]:
df_weather_matrix_v2 = df_weather_matrix_v1.drop(['is_alarm'],axis=1 ).fillna(method="ffill")

In [32]:
pd.set_option('display.max_columns', None)

In [33]:
df_weather_matrix_v2.sample(5)

,day_tempmax,day_tempmin,day_temp,day_dew,day_humidity,day_precip,day_precipcover,day_solarradiation,day_solarenergy,day_uvindex,hour_temp,hour_humidity,hour_dew,hour_precip,hour_precipprob,hour_snow,hour_snowdepth,hour_windgust,hour_windspeed,hour_winddir,hour_pressure,hour_visibility,hour_cloudcover,hour_solarradiation,hour_uvindex,hour_severerisk,region_id
16665,6.8,-4.8,0.6,-7.5,56.1,0.0,0.00,223.0,19.3,6.0,2.3,58.53,-5.0,0.0,0.0,0.0,0.0,12.6,4.7,6.0,1039.0,24.1,0.0,241.0,0.0,10.0,11
145454,0.1,-11.3,-7.6,-13.1,66.2,0.0,0.00,72.6,6.4,3.0,-11.2,79.12,-14.1,0.0,0.0,0.0,0.0,21.6,10.8,109.9,1032.0,24.1,0.0,0.0,0.0,10.0,8
172531,25.6,12.6,18.7,12.6,69.5,0.2,8.33,284.5,24.6,9.0,24.2,49.59,13.0,0.1,100.0,0.0,0.0,25.6,11.5,216.1,1015.0,14.3,79.1,772.0,8.0,10.0,19
192243,20.3,8.2,13.2,7.4,68.7,0.0,0.00,108.8,9.4,4.0,9.2,73.45,4.7,0.0,0.0,0.0,0.0,19.1,10.4,182.5,1029.0,24.1,99.5,0.0,0.0,10.0,3
51753,15.6,0.5,8.0,-3.2,48.7,0.0,0.00,198.5,17.3,7.0,10.9,40.23,-2.1,0.0,0.0,0.0,0.0,15.5,3.6,10.0,1037.6,20.0,0.0,600.0,6.0,10.0,10


In [34]:
df_weather_matrix_v2_csr = scipy.sparse.csr_matrix(df_weather_matrix_v2.values)


### TD-IDF

In [35]:
tf_idf = pickle.load(open ("/Users/bober/Desktop/study/naukma/prog/td/tfidf_transformer_v1.pkl", "rb"))
cv = pickle.load(open ("/Users/bober/Desktop/study/naukma/prog/td/count_vectorizer_v1.pkl", "rb"))

In [36]:
df_isw_upd['isw_report_text_lemm']

1       russian forc enter outskirt kyiv west bank dn...
2       russian militari like recogn initi expect lim...
3       russian militari reorgan militari effort atte...
4       russian forc complet reinforc resuppli troop ...
5       russian forc resum offens oper support envelo...
                             ...                        
329     ukrainian defens bakhmut like strateg sound e...
330     isw publish abbrevi campaign updat today janu...
331     ukrainian intellig ass russian forc prepar of...
332     coalit nato member state reportedli send ukra...
333     russian forc may engag limit spoil attack acr...
Name: isw_report_text_lemm, Length: 333, dtype: object

In [37]:
type(tf_idf)

sklearn.feature_extraction.text.TfidfTransformer

In [38]:
cv_vector = cv.transform(df_all['isw_report_text_lemm'].values.astype('U'))


In [39]:
tf_idf_vector = tf_idf.transform(cv_vector)

In [40]:
tf_idf_vector.shape

(194532, 6504)

In [41]:
df_all_data_csr = scipy.sparse.hstack((df_weather_matrix_v2_csr, tf_idf_vector), format='csr')

In [42]:
df_all_data_csr

<194532x6531 sparse matrix of type '<class 'numpy.float64'>'
	with 116130457 stored elements in Compressed Sparse Row format>

# TRAIN

In [46]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_decomposition import PLSCanonical

In [ ]:
#ts_cv = TimeSeriesSplit(
 #   n_splits=5,
 #   gap=48,
 #   max_train_size=None,
 #   test_size=40000,
#)
#array = ts_cv.split(df_all_data_csr,is_alarm)


In [47]:
def get_param(matrix):
    TP = matrix[0,0]
    FP = matrix[0,1]
    FN = matrix[1,0]
    TN = matrix[1,1]
    recall = TP/(TP+FN)
    precision = TP/(TP+FP)
    accuracy = (TP+TN)/(TP+FP+TN+FN)
    result = f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}'
    return result

In [48]:
RANDOM_STATE = 1
X_train, X_test, y_train, y_test = train_test_split(df_all_data_csr, is_alarm,
test_size=0.20,
random_state = RANDOM_STATE,
shuffle=True)

# 1 too long

In [37]:
#model_1 = svm.SVC()
#model_1.fit(X_train, y_train)

In [38]:
#predicted1 = model_1.predict(X_test)
#np.mean(predicted1 == y_test)

# 2 

In [39]:
model_2 =  LogisticRegression(penalty="l1", tol=0.01, solver="saga")
model_2.fit(X_train, y_train)

LogisticRegression(penalty='l1', solver='saga', tol=0.01)

In [43]:
predicted2 = model_2.predict(X_test)
np.mean(predicted2 == y_test)

0.7463181432647081

In [61]:
confusion_matrix_2 = confusion_matrix(y_test, predicted2)
confusion_matrix_2

array([[28832,   167],
       [ 9703,   205]])

In [62]:
get_param(confusion_matrix_2)

'Accuracy: 0.7463181432647081, Recall: 0.7482029323991177, Precision: 0.99424118073037'

# 3 done

In [49]:
model_3 = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
model_3.fit(X_train, y_train)

/Users/bober/Desktop/study/naukma/prog/pyenv/lib/python3.9/site-packages/sklearn/linear_model/_stochastic_gradient.py:702: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDClassifier(max_iter=5)

In [50]:
predicted3 = model_3.predict(X_test)
np.mean(predicted3 == y_test)

0.7341866502171845

In [53]:
confusion_matrix_3 = confusion_matrix(y_test, predicted3)
confusion_matrix_3

array([[27071,  1928],
       [ 8414,  1494]])

In [54]:
get_param(confusion_matrix_3)

'Accuracy: 0.7341866502171845, Recall: 0.762885726363252, Precision: 0.9335149487913377'

# 4 done

In [ ]:
model_4 = RandomForestClassifier()
model_4.fit(X_train, y_train)

In [ ]:
predicted4 = model_4.predict(X_test)
np.mean(predicted4 == y_test)

In [ ]:
confusion_matrix_4 = confusion_matrix(y_test, predicted4)
confusion_matrix_4

In [ ]:
get_param(confusion_matrix_4)

# 5 done

In [ ]:
model_5 = tree.DecisionTreeClassifier()
model_5.fit(X_train, y_train) 

In [ ]:
predicted5 = model_5.predict(X_test)
np.mean(predicted5 == y_test)

In [ ]:
confusion_matrix_5 = confusion_matrix(y_test, predicted5)
confusion_matrix_5

In [ ]:
get_param(confusion_matrix_5)

# 6

In [60]:
model_6 = svm.SVR()
model_6.fit(X_train, y_train)


SyntaxError: invalid syntax (2166845547.py, line 1)

In [51]:
predicted6 = model_6.predict(X_test)
np.mean(predicted6 == y_test)

NotFittedError: This PLSCanonical instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
confusion_matrix_6 = confusion_matrix(y_test, predicted6)
confusion_matrix_6

In [ ]:
get_param(confusion_matrix_6)

## Saving model

In [ ]:
INPUT_FOLDER = 'training_models'

In [ ]:
name = 'model_5'
model = model_5
with open(f'{INPUT_FOLDER}/{name}.pkl','wb') as handle:
    pickle.dump(model, handle)

## Calculating a confusion matrix

In [ ]:
confusion_matrix(y_true, y_pred)